# Text Models

* **Name:** Aldo Barriente
* **Course:** DS 5001
* **Instructor:** Professor Rafael Alvarado

## Set up

In [14]:
data_in = './data_in'
data_out = './data_out'
data_prefix = 'zapatistas'
OHCO = ['title_name', 'section_num', 'para_num', 'sent_num', 'token_num']

In [15]:
import pandas as pd
import re

In [16]:
lines = ''
with open(f'{data_in}/{data_prefix}-ch1.txt', 'r', encoding='utf8') as chap_one:
    lines = chap_one.readlines()

## Creating dataframe

In [17]:
text = pd.DataFrame(lines, columns=['line_str'])
text.columns = ['line_str']
text.index.name = 'line_num'
text.line_str = text.line_str.str.strip()

## Extracting title and removing cruft 

In [18]:
begin = text.line_str.str.match(r"Chapter \d+:")
end = text.line_str.str.match(r"LANIC Home | Zapatistas!")

In [19]:
begin_index = text.loc[begin].index[0]
end_index = text.loc[end].index[0]

In [20]:
text = text.loc[begin_index - 1 : end_index - 1]

In [21]:
title = text.loc[begin_index].line_str

In [22]:
title = re.sub(r'Chapter \d+:\s', '', title).strip()

## Chunking and group by title

In [23]:
text[OHCO[:1]] = title

In [24]:
text.loc[100]

line_str      Chapter 1: The Revolt
title_name               The Revolt
Name: 100, dtype: object

## Chunking by section

In [39]:
section_lines = text.line_str.str.match(r"$")

In [40]:
text.loc[section_lines]

,line_str,title_name
line_num,,
99,,The Revolt
101,,The Revolt
206,,The Revolt
248,,The Revolt
249,,The Revolt
251,,The Revolt
285,,The Revolt
309,,The Revolt
332,,The Revolt


In [41]:
section_num = [i+1 for i in range(text.loc[section_lines].shape[0])]

In [42]:
text.loc[section_lines, 'section_num'] = section_num

In [44]:
text.loc[section_lines]

,line_str,title_name,section_num
line_num,,,
99,,The Revolt,1.0
101,,The Revolt,2.0
206,,The Revolt,3.0
248,,The Revolt,4.0
249,,The Revolt,5.0
251,,The Revolt,6.0
285,,The Revolt,7.0
309,,The Revolt,8.0
332,,The Revolt,9.0


In [45]:
text.section_num = text.section_num.ffill()

In [47]:
text.loc[240:260]

,line_str,title_name,section_num
line_num,,,
240,against our class enemies.,The Revolt,3.0
241,In this first issue we present our Declaration...,The Revolt,3.0
242,"Army, and we publish the orders to be followed...",The Revolt,3.0
243,of the EZLN in our advance through national te...,The Revolt,3.0
244,are the Revolutionary Laws that will be enacte...,The Revolt,3.0
245,in order to guarantee their revolutionary cont...,The Revolt,3.0
246,so we can begin the process of building a new ...,The Revolt,3.0
247,To live for our country or die for freedom.,The Revolt,3.0
248,,The Revolt,4.0


### Section number clean up

In [48]:
text = text.loc[~text.section_num.isna()]
text = text.loc[~section_lines]
text.section_num = text.section_num.astype('int')

## Grouping lines by title and section number

In [52]:
text_section = text.groupby(OHCO[:2]).line_str.apply(lambda x: '\n'.join(x)).to_frame()
text_section['line_str'] = text_section.line_str.str.strip()

In [53]:
text_section.head(10)

line_str
title_name section_num                                                   
The Revolt 1                                        Chapter 1: The Revolt
           2            [The Mexican Awakener [El Despertador Mexicano...
           3            Editorial\nMexicans: workers, campesinos, stud...
           5                                           Revolutionary Laws
           6            Women's Revolutionary Law\nIn the just fight f...
           7            Urban Reform Law\nIn the urban zones controlle...
           8            Labor Law: Additions to the Present Law\nThe f...
           9            Industry and Commerce Law\nFirst: The prices o...
           10           Social Security Law\nFirst: Abandoned children...
           11           Justice Law\nFirst: All prisoners in all priso...

## Splitting into paragraphs

In [57]:
text_para = text_section['line_str'].str.split(r'\n\n+', expand=True).stack().to_frame().rename(columns={0:'para_str'})
text_para.index.names = OHCO[:3]

In [58]:
text_para.head()

para_str
title_name section_num para_num                                                   
The Revolt 1           0                                     Chapter 1: The Revolt
           2           0         [The Mexican Awakener [El Despertador Mexicano...
           3           0         Editorial\nMexicans: workers, campesinos, stud...
           5           0                                        Revolutionary Laws
           6           0         Women's Revolutionary Law\nIn the just fight f...

In [59]:
text_para['para_str'] = text_para['para_str'].str.replace(r'\n', ' ').str.strip()
text_para = text_para[~text_para['para_str'].str.match(r'^\s*$')]

In [60]:
text_para.head(10)

para_str
title_name section_num para_num                                                   
The Revolt 1           0                                     Chapter 1: The Revolt
           2           0         [The Mexican Awakener [El Despertador Mexicano...
           3           0         Editorial Mexicans: workers, campesinos, stude...
           5           0                                        Revolutionary Laws
           6           0         Women's Revolutionary Law In the just fight fo...
           7           0         Urban Reform Law In the urban zones controlled...
           8           0         Labor Law: Additions to the Present Law The fo...
           9           0         Industry and Commerce Law First: The prices of...
           10          0         Social Security Law First: Abandoned children ...
           11          0         Justice Law First: All prisoners in all prison...

## Splitting into sentences

In [61]:
sent_pat = r'[.?!;:"]+'
text_sent = text_para['para_str'].str.split(sent_pat, expand=True)\
    .stack().to_frame().rename(columns={0:'sent_str'})
text_sent.index.names = OHCO[:4]
text_sent = text_sent[~text_sent['sent_str'].str.match(r'^\s*$')]

In [62]:
text_sent.head(10)

sent_str
title_name section_num para_num sent_num                                                   
The Revolt 1           0        0                                                 Chapter 1
                                1                                                The Revolt
           2           0        0         [The Mexican Awakener [El Despertador Mexicano...
                                1          It was the first document released by the Zap...
                                2          It contains their Declaration of War, an edit...
                                3         ] El Despertador Mexicano Declaration of War [...
                                4                                   TO THE PEOPLE OF MEXICO
                                5                              MEXICAN BROTHERS AND SISTERS
                                6               We are the product of 500 years of struggle
                                7          first against slavery, then during the War of...

## Splitting into tokens

In [63]:
token_pat = r"[\s',-]+"
text_token = text_sent['sent_str'].str.split(token_pat, expand=True)\
    .stack().to_frame().rename(columns={0:'token_str'})
text_token.index.names = OHCO[:5]

In [64]:
text_token.sample(50)

token_str
title_name section_num para_num sent_num token_num                
The Revolt 2           0        18       32                Article
           14          0        2        19                     of
           2           0        11       10                    our
                                10       4                    work
           11          0        1        18                     of
           12          0        29       17                product
           19          0        215      25                    you
           12          0        5        8          agro/livestock
           3           0        4        2                hundreds
           15          0        30       20                  goods
           2           0        23       6                      we
           19          0        137      19                     in
           16          0        36       0                        
           13          0        19       2                    same
           19          0        184      1                      We
                                15       7                     and
                                1        4                     any
           2           0        33       0                        
           13          0        25       7                      of
           2           0        46       13                    the
           19          0        236      7                     for
           11          0        3        36                  guilt
           6           0        3        1                   Women
           19          0        193      2                       *
           2           0        2        3                   their
           9           0        2        11              merchants
           19          0        210      1                    That
           2           0        48       0                        
           15          0        9        11               civilian
                                32       56                    the
           12          0        40       23              polluting
           16          0        38       6                     war
           13          0        12       3                      of
           3           0        24       6                 present
                                15       18                    the
           19          0        109      1                      We
           13          0        39       3                     not
           19          0        234      4                   until
           13          0        45       21                     of
           16          0        10       0                        
                                26       3                    take
           19          0        251      11                    get
           2           0        48       4                     and
           3           0        22       2             Despertador
           12          0        40       29                 misuse
           15          0        12       5                   taxes
           2           0        49       14                   many
           16          0        22       1                      To
           19          0        223      16                 orders
           12          0        9        9                     the